In [ ]:
import pandas as pd 
df=pd.read_csv('analysis_id(in).csv') #creating dataframe from my csv file

In [ ]:
df.info() # basic statistical EDA on data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              5000 non-null   int64  
 1   date_hour               5000 non-null   object 
 2   client                  5000 non-null   object 
 3   device_type             5000 non-null   object 
 4   time_zone               5000 non-null   object 
 5   os_name                 5000 non-null   object 
 6   browser_name            5000 non-null   object 
 7   ad_unit                 5000 non-null   object 
 8   size                    5000 non-null   object 
 9   bidder                  5000 non-null   object 
 10  bid_range               5000 non-null   object 
 11  media_type              5000 non-null   object 
 12  request_count           5000 non-null   int64  
 13  response_count          5000 non-null   int64  
 14  prebid_win_count        1167 non-null   

In [ ]:
b_bidder=df['bidder'].value_counts() # checking value distribution of bidder
b_bidder

bidder
                1210
ix               464
onemobile        407
medianet         384
criteo           306
pubmatic         298
rhythmone        246
33across         240
triplelift       232
appnexus         197
openx            197
rubicon          178
emx_digital      169
sharethrough     139
pulsepoint       134
sovrn            102
undertone         49
teads             48
Name: count, dtype: int64

In [ ]:
a=df[df['bidder']==' '].index # storing the index of those entries that has blank bidder
a

Index([   3,    6,   11,   12,   14,   16,   19,   24,   29,   30,
       ...
       4981, 4983, 4984, 4991, 4992, 4993, 4995, 4997, 4998, 4999],
      dtype='int64', length=1210)

In [ ]:
df=df.drop(index=a)  # dropping those entries that has blank bidder

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3790 entries, 0 to 4996
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              3790 non-null   int64  
 1   date_hour               3790 non-null   object 
 2   client                  3790 non-null   object 
 3   device_type             3790 non-null   object 
 4   time_zone               3790 non-null   object 
 5   os_name                 3790 non-null   object 
 6   browser_name            3790 non-null   object 
 7   ad_unit                 3790 non-null   object 
 8   size                    3790 non-null   object 
 9   bidder                  3790 non-null   object 
 10  bid_range               3790 non-null   object 
 11  media_type              3790 non-null   object 
 12  request_count           3790 non-null   int64  
 13  response_count          3790 non-null   int64  
 14  prebid_win_count        1167 non-null   float

In [ ]:
total_bid=df.groupby('bidder')['request_count'].sum().astype(float).reset_index(name='total_bid') # creating a dataframe to store sum of all bids put by each bidder
total_bid

,bidder,total_bid
0,33across,886.0
1,appnexus,830.0
2,criteo,3181.0
3,emx_digital,393.0
4,ix,3051.0
5,medianet,2556.0
6,onemobile,2726.0
7,openx,1165.0
8,pubmatic,2335.0
9,pulsepoint,191.0


In [ ]:
won_bid=df.groupby('bidder')['prebid_win_count'].sum().reset_index(name='won_bid') # creating a dataframe to store sum of all won bids put by each bidder
won_bid

,bidder,won_bid
0,33across,136.0
1,appnexus,175.0
2,criteo,587.0
3,emx_digital,56.0
4,ix,184.0
5,medianet,253.0
6,onemobile,759.0
7,openx,86.0
8,pubmatic,255.0
9,pulsepoint,71.0


In [ ]:
merged=pd.merge(total_bid,won_bid,on='bidder',how='inner') # merging won bid count df and total bid count df
merged

,bidder,total_bid,won_bid
0,33across,886.0,136.0
1,appnexus,830.0,175.0
2,criteo,3181.0,587.0
3,emx_digital,393.0,56.0
4,ix,3051.0,184.0
5,medianet,2556.0,253.0
6,onemobile,2726.0,759.0
7,openx,1165.0,86.0
8,pubmatic,2335.0,255.0
9,pulsepoint,191.0,71.0


In [ ]:
merged['win_rate']=merged['won_bid']/merged['total_bid'] # calculating win rate
merged=merged.sort_values(by='win_rate',ascending=False)
merged.reset_index(drop=True,inplace=True)
merged # final dataframe that contains all the bidder their total bids over the time placed and total bid won and their win rate

,bidder,total_bid,won_bid,win_rate
0,undertone,104.0,52.0,0.500000
1,teads,700.0,323.0,0.461429
2,pulsepoint,191.0,71.0,0.371728
3,sharethrough,332.0,104.0,0.313253
4,onemobile,2726.0,759.0,0.278430
5,rhythmone,655.0,160.0,0.244275
6,appnexus,830.0,175.0,0.210843
7,criteo,3181.0,587.0,0.184533
8,33across,886.0,136.0,0.153499
9,emx_digital,393.0,56.0,0.142494


In [38]:
print(f"The bidder having highest win_rate is {merged.iloc[0]}")

The bidder having highest win_rate is bidder       undertone
total_bid        104.0
won_bid           52.0
win_rate           0.5
Name: 0, dtype: object
